### Google News Scrapper

This piece of code is used to scrape news articles by searching on http://www.news.google.com with keywords of interest and specific date ranges. We generate a csv files of news text

In [140]:
%config IPCompleter.greedy=True

In [5]:
pip install newspaper3k

     |████████████████████████████████| 211 kB 1.7 MB/s eta 0:00:01
     |████████████████████████████████| 7.4 MB 3.2 MB/s eta 0:00:01
     |████████████████████████████████| 80 kB 2.4 MB/s eta 0:00:01
     |████████████████████████████████| 87 kB 3.8 MB/s eta 0:00:01
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3357 sha256=7025af686ab2faaff7bf0b29424dd3b9daf182171a75f4664323e01fadbb5c63
  Stored in directory: /Users/hitman94/Library/Caches/pip/wheels/b6/09/68/a9f15498ac02c23dde29f18745bc6a6f574ba4ab41861a3575
  Created wheel for jieba3k: filename=jieba3k-0.35.1-py3-none-any.whl size=7398407 sha256=086e04880362a168846ec07efd6591be6c8774c208f2b455e3ff355af1316816
  Stored in directory: /Users/hitman94/Library/Caches/pip/wheels/1f/7e/0c/54f3b0f5164278677899f2db08f2b07943ce2d024a3c862afb
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13536 sha256=32abd67bc7334257f29ad74963533d6f4e14233e50eb9a524d0e152ab73aa065
  St

In [1]:
import requests
from bs4 import BeautifulSoup
from newspaper import Article
from GoogleNews import GoogleNews
from newspaper import Config
import nltk
import datetime
import pandas as pd
import time
import os
import numpy as np

In [2]:
#URL below searches google news based on two keywords "bitcoin" and "cryptocurrency"
URL = "https://www.google.com/search?q=bitcoin+cryptocurrency&hl=en&gl=us&as_drrb=b&tbas=0&tbs=cdr:1,cd_min:{minDate},cd_max:{maxDate},sbd:1&tbm=nws&sxsrf=ACYBGNRfmviSo9arK1e_P_YIl5wsskZBPw:1574225634362&source=lnt&sa=X&ved=0ahUKEwj4wu29__flAhWV9Z4KHaKJAGcQpwUIIA&biw=1685&bih=863&dpr=1.1"

In [3]:
#config will allow us to access the specified url for which we are not authorized. Sometimes we may get 403 
#client error while parsing #the link to download the article.

nltk.download('punkt')

"""headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/46.0.2490.80 Safari/537.36',
    'Content-Type': 'text/html',
}"""

user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'

config = Config()
config.browser_user_agent = user_agent

[nltk_data] Downloading package punkt to /Users/hitman94/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
maxCount = 9  # max 10 news articles per day
dateFormat = '%m/%d/%Y'
newsColumns = ['index', 'date', 'news_1_url', 'news_1_text',
             'news_2_url', 'news_2_text', 'news_3_url', 'news_3_text',
             'news_4_url', 'news_4_text', 'news_5_url', 'news_5_text',
             'news_6_url', 'news_6_text', 'news_7_url', 'news_7_text',
             'news_8_url', 'news_8_text', 'news_9_url', 'news_9_text',
             'news_10_url', 'news_10_text']

In [5]:
def runGoogleNewsScrapper(**params):
    outputFileName = ''
    for key, value in params.items():
        if key == 'minDate':
            minDate = value
        if key == 'outputFile':
            outputFileName = value
    
    googlenews=GoogleNews(start=minDate,end=minDate)
    googlenews.search('Bitcoin Cryptocurrency')
    result=googlenews.result()
    df=pd.DataFrame(result)

    newsDataDictionary = dict() #creating a new dictionary
    columns = []
    newsDataDictionary['date'] = minDate
    columns.append('date')
    """response = requests.get(URL.format(**params), headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    newsDataDictionary['status_code'] = response.status_code
    columns.append('status_code')
    if response.status_code != 200:
        print("******** fail ********** ")
        return"""
    # print(response.url)
    #newsDataDictionary['url'] = response.url
    #columns.append('url')
    
    #listofLinkStr = []
    count = 1
   
    """for link in soup.find_all('a'):
        linkStr = str(link.get('href'))
        try:
            #linkStr.startswith("https://") and linkStr.find('google.com') == -1 and linkStr.find(
                    #"https://www.youtube.com/") == -1 and linkStr.find("https://wwwblogger..com/") == -1
            if linkStr.startswith("/url?q=https://"):
                
                #linkStrConverisonToList = list(linkStr)
                
                #code to extraxt href link from the soup instance
                linkStrList = linkStr.split('&')
                slicedList = list(linkStrList[0])
                del slicedList[:7]
                #del slicedList[-4:]
                listToStr = ' '.join(map(str, slicedList)) 
                linkStr = listToStr.replace(" ","")
                
                if linkStr in listofLinkStr:
                    print("")
                else:
                    listofLinkStr.append(linkStr)

                article = Article(linkStr)
                article.download()
                article.parse()
                print(linkStr)
                #print(article.authors)
                #print(article.publish_date)
                #print(article.text)
                newsCount = 'news_' + str(count)
                newsDataDictionary[newsCount + '_url'] = linkStr
                newsDataDictionary[newsCount + '_text'] = article.text
                newsDataDictionary[newsCount + '_publish_date'] = article.publish_date

                columns.append(newsCount + '_url')
                columns.append(newsCount + '_text')
                columns.append(newsCount + '_publish_date')
                count += 1
                if count >= maxCount:
                    break
        except:
            pass"""
        
    """print("----------------------------------------------------------------------------------------")
    for x in range(len(listofLinkStr)): 
        print(listofLinkStr[x]) 
    print("----------------------------------------------------------------------------------------")"""
    
    for ind in df.index:
        article = Article(df['link'][ind],config=config)
        try:
            article.download()
            article.parse()
        except:
            print('***FAILED TO DOWNLOAD***', article.url)
            continue
        article.nlp()
        newsCount = 'news_' + str(count)
        newsDataDictionary[newsCount + '_url'] = df['link'][ind]
        newsDataDictionary[newsCount + '_title']=article.title
        newsDataDictionary[newsCount + '_text']=article.text
        #list.append(dict)
        
        columns.append(newsCount + '_url')
        columns.append(newsCount + '_title')
        columns.append(newsCount + '_text')
        

        count += 1
        if count >= maxCount:
            break

    #news_df=pd.DataFrame(list)
    #news_df.to_csv("/Volumes/Sashank Work/MSc DCU/Semester 2/CA683 Data Analytics and Data Mining/Assignment/Dataset/Google News/articles.csv")
    
    """ctr = 1
    for linkStr in listofLinkStr:
        article = Article(linkStr)
        
        try:
            article.download()
            article.parse()
        except:
            print('***FAILED TO DOWNLOAD***', article.url)
            continue
            
        #article.download()
        #article.parse()
        
        #print(article.authors)
        #print(article.publish_date)
        #print(article.text)
        newsCount = 'news_' + str(ctr)
        newsDataDictionary[newsCount + '_url'] = linkStr
        newsDataDictionary[newsCount + '_text'] = article.text
        #newsDataDictionary[newsCount + '_publish_date'] = article.publish_date

        columns.append(newsCount + '_url')
        columns.append(newsCount + '_text')
        #columns.append(newsCount + '_publish_date') 
        
        ctr += 1
        if ctr >= maxCount:
                break"""
    
    newsDataDf = pd.DataFrame(newsDataDictionary, index=[0],columns=newsColumns)
    
    """
    if os.path.exists('/Volumes/Sashank Work/MSc DCU/Semester 2/CA683 Data Analytics and Data Mining/Assignment/Dataset/Google News/' + outputFileName):
        keep_header = False
        news_data_df.to_csv('/Volumes/Sashank Work/MSc DCU/Semester 2/CA683 Data Analytics and Data Mining/Assignment/Dataset/Google News/' + outputFileName, mode='a', header=keep_header)
    else:
        keep_header = True
        news_data_df.to_csv('/Volumes/Sashank Work/MSc DCU/Semester 2/CA683 Data Analytics and Data Mining/Assignment/Dataset/Google News/' + outputFileName)"""
        
    # if file does not exist write header 
    if not os.path.isfile('/Volumes/Sashank Work/MSc DCU/Semester 2/CA683 Data Analytics and Data Mining/Assignment/Dataset/Google News/' + outputFileName):
        newsDataDf.to_csv('/Volumes/Sashank Work/MSc DCU/Semester 2/CA683 Data Analytics and Data Mining/Assignment/Dataset/Google News/' + outputFileName)
    else: # else it exists so append without writing the header
        newsDataDf.to_csv('/Volumes/Sashank Work/MSc DCU/Semester 2/CA683 Data Analytics and Data Mining/Assignment/Dataset/Google News/' + outputFileName, mode='a', header=False)

    
    #news_data_df.to_csv('/Volumes/Sashank Work/MSc DCU/Semester 2/CA683 Data Analytics and Data Mining/Assignment/Dataset/Google News/' + outputFileName)
    return newsDataDictionary

In [6]:
#no issues in this code
def googleNewsScrapper(startDate, endDate, outputFileName):
    stepObj = datetime.timedelta(days=1)
    startDateTimeObj = datetime.datetime.strptime(startDate, dateFormat)
    endDateTimeObj = datetime.datetime.strptime(endDate, dateFormat)

    while startDateTimeObj <= endDateTimeObj:
        startDate = startDateTimeObj.strftime(dateFormat)
        print(startDate)
        runGoogleNewsScrapper(minDate=startDate, maxDate=startDate, outputFile=outputFileName)
        time.sleep(np.random.randint(2, 5))
        startDateTimeObj += stepObj

In [7]:
def sortNewsReport(inputFileName, cleanedOutputFileName, saveIndex=False):
    df = pd.read_csv('/Volumes/Sashank Work/MSc DCU/Semester 2/CA683 Data Analytics and Data Mining/Assignment/Dataset/Google News/' + inputFileName)
    df = df.set_index('date', drop=True)
    df.index = pd.to_datetime(df.index)
    df = df.sort_index().drop_duplicates(keep='first')
    df.to_csv('/Volumes/Sashank Work/MSc DCU/Semester 2/CA683 Data Analytics and Data Mining/Assignment/Dataset/Google News/' + cleanedOutputFileName)
    if saveIndex:
        df_i = pd.DataFrame(df.index)
        df_i.to_csv('/Volumes/Sashank Work/MSc DCU/Semester 2/CA683 Data Analytics and Data Mining/Assignment/Dataset/Google News/' + cleanedOutputFileName[0:-4] + '_index.csv')

In [8]:
def cleanNewsReport(inputFileName, cleanedOutputFileName, saveIndex=False):
    ctr = 1
    final_df = pd.read_csv('/Volumes/Sashank Work/MSc DCU/Semester 2/CA683 Data Analytics and Data Mining/Assignment/Dataset/Google News/' + inputFileName)
    
    # get only given columns
    final_df = final_df[['date', 'news_1_text', 'news_2_text', 'news_3_text', 'news_4_text', 'news_5_text', 'news_6_text',
             'news_7_text', 'news_8_text', 'news_9_text', 'news_10_text']]
    final_df = final_df.set_index('date', drop=True)
    final_df.index = pd.to_datetime(final_df.index, format=dateFormat)
    # sort and drop duplicates
    final_df = final_df.sort_index().drop_duplicates(keep='first')
    index = np.unique(final_df.index, return_index=True)[1]
    final_df = final_df.iloc[index]
    final_df.to_csv('/Volumes/Sashank Work/MSc DCU/Semester 2/CA683 Data Analytics and Data Mining/Assignment/Dataset/Google News/' + cleanedOutputFileName)
    if saveIndex:
        df_i = pd.DataFrame(final_df.index)
        df_i.to_csv('/Volumes/Sashank Work/MSc DCU/Semester 2/CA683 Data Analytics and Data Mining/Assignment/Dataset/Google News/' + cleanedOutputFileName[0:-4] + '_index.csv')

In [10]:
#no issues in this code
if __name__ == "__main__":
    startDate = '03/01/2021'
    # endDate = datetime.datetime.now().strftime(dateFormat)
    endDate = '03/20/2021'
    newsFilename = 'google_news_final_test.csv'

    googleNewsScrapper(startDate, endDate, newsFilename)
    newsCleanedFilename = newsFilename[0:-4] + '_cleaned.csv'
    cleanNewsReport(newsFilename, newsCleanedFilename)
    #testFunction(minDate = startDate, maxDate = endDate)

03/01/2021
03/02/2021
***FAILED TO DOWNLOAD*** https://www.nasdaq.com/articles/what-is-bitcoin-and-how-does-it-work-2021-03-02
03/03/2021
03/04/2021
***FAILED TO DOWNLOAD*** https://money.usnews.com/investing/cryptocurrency/articles/why-everyone-should-own-some-cryptocurrency
***FAILED TO DOWNLOAD*** https://seekingalpha.com/article/4411554-case-against-cryptocurrencies
03/05/2021
***FAILED TO DOWNLOAD*** https://www.washingtonpost.com/business/2021/03/05/john-mcafee-charged-cryptocurrency/
03/06/2021
03/07/2021
03/08/2021
***FAILED TO DOWNLOAD*** https://seekingalpha.com/news/3670206-bitcoin-roundup-back-over-50k-post-stimulus-norways-aker-wants-in
03/09/2021
03/10/2021
03/11/2021
03/12/2021
***FAILED TO DOWNLOAD*** https://mooresvilletribune.com/business/investment/personal-finance/will-buying-bitcoin-impact-my-tax-return/article_f188a792-2819-556c-b762-fe1415aedc87.html
03/13/2021
***FAILED TO DOWNLOAD*** https://www.bangkokpost.com/business/2083083/bitcoin-surging-toward-60-000
03/

In [75]:
#testing code
from pandas import DataFrame
final_df_list = [5,2,6,2,7,5,6,8,2,9]
test_index = np.unique(final_df_list, return_index=True)[1]
print(test_index)
df = DataFrame(final_df_list,columns=['Column_Name'])
df = df.iloc[test_index]
print(df)

[1 0 2 4 7 9]
   Column_Name
1            2
0            5
2            6
4            7
7            8
9            9


In [96]:
#testing code
def testFunction(**params):
    response = requests.get(URL.format(**params), headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    print(soup)

In [269]:
df_read = pd.read_csv("/Volumes/Sashank Work/MSc DCU/Semester 2/CA683 Data Analytics and Data Mining/Assignment/Dataset/Google News/google_news_final_cleaned.csv")

In [270]:
df_read.head

<bound method NDFrame.head of          date                                        news_1_text  \
0  2021-03-18  eToro's CEO speculates on what’s driving the c...   

                                         news_2_text  \
0  SINGAPORE: Amid Singapore's push for sustainab...   

                                         news_3_text  \
0  Join legendary investor Matt McCall on March 2...   

                                         news_4_text  \
0  (REUTERS) - Accounting for and storing highly ...   

                                         news_5_text  news_6_text  \
0  How bitcoin is consider the magnetic cryptocur...          NaN   

   news_7_text  news_8_text  news_9_text  news_10_text  
0          NaN          NaN          NaN           NaN  >

In [271]:
link = "https://investorplace.com/moneywire/2021/03/bitcoin-btc-not-laughing-at-my-bold-cryptocurrency-prediction/"
newsDictionary = dict() #creating a new dictionary
cutter = 1
article = Article(link)
article.download()
article.parse()

#print(article.authors)
#print(article.publish_date)
#print(article.text)
newsCounter = 'news_' + str(cutter)
newsDictionary[newsCounter + '_url'] = link
newsDictionary[newsCounter + '_text'] = article.text

In [272]:
newsDictionary

{'news_1_url': 'https://investorplace.com/moneywire/2021/03/bitcoin-btc-not-laughing-at-my-bold-cryptocurrency-prediction/',
 'news_1_text': 'Join legendary investor Matt McCall on March 24 when he unveils how a new investment is set to bring Wall Street’s wealth to the everyday American.'}